# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import pyodbc
import os
import warnings

# Constants

In [2]:
# Define the years for each dataframe
years = [2017, 2018, 2019, 2021, 2022]

# Intialize Useful Functions

In [3]:
def safe_convert(val):
    try:
        return int(val)
    except ValueError:
        print(f"Value {val} can't be converted to int")
        return None
    
def import_mdb(MDBs, DRV, PWD, NAMES):
    
    databases = {}
    
    for MDB, NAME in zip(MDBs, NAMES):
        # connect to db
        con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB,PWD))
        cur = con.cursor()

        # List all tables in the database
        tables = list(map(lambda t: t.table_name, con.cursor().tables(tableType='TABLE')))

        # Initialize an empty dictionary to hold your dataframes and databases
        database = {}

        # Try to read each table one by one
        for table in tables:
            try:
                df = pd.read_sql(f'SELECT * FROM [{table}]', con)  # enclose table name in brackets
                database[table] = df
                print(f"Successfully read table: {table}")
            except Exception as e:
                print(f"Failed to read table: {table}")
                print(f"Error: {e}")
        databases[NAME] = database
        
    return databases

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', 'pandas only support SQLAlchemy connectable.*')
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

# Import Main Data

In [4]:
# Main Data Filepaths WINDOWS
# MAIN_PATH = [
#                '../data/SRC2019/SRC2019.mdb;',
#                '../data/SRC2021/SRC2021.mdb;',
#                '../data/SRC2022/SRC2022.mdb;',
#               ]

# Main Data Filepaths MAC
MAIN_PATH = [
               '../../data/SRC2017/ARE2017.csv',
               '../../data/SRC2018/ARE2018.csv',
               '../../data/SRC2019/ARE2019.csv',
               '../../data/SRC2021/ARE2021.csv',
               '../../data/SRC2022/ARE2022.CSV',
              ]
MAIN_NAMES = [
               'MAIN2017',
               'MAIN2018',
               'MAIN2019',
               'MAIN2021',
               'MAIN2022',
              ]

main_data = {}
for name, filepath in zip(MAIN_NAMES, MAIN_PATH):
    main_data[name] = {'Annual Regents Exams':pd.read_csv(filepath, thousands=',')}

# Main data Windows
# main_data = import_mdb(MAIN_PATH, DRV, PWD, MAIN_NAMES)

# Import Enrollment Data

In [5]:
# DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
# PWD = 'pw'

# # Enroll Data Filepaths
# ENROLL_PATH = [
#                '../data/enrollment_2019/ENROLL2019.mdb;',
#                '../data/enrollment_2021/ENROLL2021.mdb;',
#                '../data/enrollment_2022/ENROLL2022.mdb;',
#               ]
# ENROLL_NAMES = [
#                'ENROLL2019',
#                'ENROLL2021',
#                'ENROLL2022',
#               ]


# enroll_data = import_mdb(ENROLL_PATH, DRV, PWD, ENROLL_NAMES)

# Import Dropout Rates

In [6]:
data22 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2022.csv", thousands=',')
data21 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2021.csv", thousands=',')
data19 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2019.csv", thousands=',')

dropout_dfs = [data19, data21, data22]

# Combine Dropout Data

In [7]:
# Remove districts, only keep schools
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i] = df[df['aggregation_type'] == 'School']

#   Only keep schools which are present in all years    #
#########################################################

# Convert the 'ID' column of each DataFrame to a set
set1 = set(dropout_dfs[0]['aggregation_code'])
set2 = set(dropout_dfs[1]['aggregation_code'])
set3 = set(dropout_dfs[2]['aggregation_code'])

# Find the intersection of all 4 sets - i.e., the common IDs
common_ids = set1 & set2 & set3

# Filter each DataFrame to only include rows with a common ID
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i] = df[df['aggregation_code'].isin(common_ids)]
    
###########################################################

common_ids = set(df['aggregation_code'])

# Initialize a list to store the updated dataframes
updated_dfs = []

# Iterate over the dropout dataframes and the years together
for year, df in zip(years, dropout_dfs):
    # Add a new column 'year' to the dataframe
    df['year'] = year
    # Append the updated dataframe to the list
    updated_dfs.append(df)

# Concatenate the updated dataframes together
dropout_df = pd.concat(updated_dfs)

# drop disttricts from the dataframe
dropout_df = dropout_df[~dropout_df['aggregation_code'].astype(str).str.endswith('0000.0')]

# Reset the index of the combined dataframe
dropout_df = dropout_df.reset_index(drop=True)

common_ids = set(dropout_df['aggregation_code'])

In [8]:
# Define a function to check if 'comparison' year is in 'membership_desc'
def check_year_in_desc(row):
    return str(row['comparison']) in row['membership_desc'] and "August" not in row['membership_desc']

dropout_df = dropout_df[dropout_df['subgroup_name'] == 'All Students']
dropout_df['report_school_year'] = dropout_df['report_school_year'].apply(lambda x: int(str(x).split('-')[1]))
dropout_df['report_school_year'] = dropout_df['report_school_year'].apply(lambda x: x + 2000 if x < 100 else x)

# Convert the 'report_school_year' to int and subtract 4
dropout_df['comparison'] = dropout_df['report_school_year'] - 4

# Apply the function to each row of dropout_df
dropout_df = dropout_df[dropout_df.apply(check_year_in_desc, axis=1)]

# Combine Main Data

### Remove Districts, Keep Schools Common Across Databases

In [9]:
common_entity_ids = set(main_data['MAIN2019']['Annual Regents Exams']['ENTITY_CD']) 

for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    current_data['ENTITY_CD'] = current_data['ENTITY_CD'].apply(safe_convert)
    current_data = current_data[~current_data['ENTITY_CD'].astype(str).str.endswith('0000')]

    common_entity_ids = set(current_data['ENTITY_CD']) & common_ids
    
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database]['Annual Regents Exams'] = current_data[current_data['ENTITY_CD'].isin(common_ids)]

In [10]:
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database] = {subject: current_data[current_data['SUBJECT'] == subject] for subject in current_data['SUBJECT'].unique()}

In [11]:
old_to_new = {
    'REG_PHYS_PS':'Regents Phy Set/Physics',
    'REG_NF_GLHIST':'Regents NF Global History',
    'REG_COMENG':'Regents Common Core English Language Art', 
    'REG_ESCI_PS':'Regents Phy Set/Earth Sci',
    'REG_CHEM_PS':'Regents Phy Set/Chemistry', 
    'REG_COMALG1':'Regents Common Core Algebra I', 
    'REG_COMGEOM':'Regents Common Core Geometry', 
    'REG_LENV':'Regents Living Environment',
    'REG_USHG_RV':"Regents US History&Gov't"
}

new_to_old = {}

for key in old_to_new:
    new_to_old[old_to_new[key]] = key

tests = (set(old_to_new[test] for test in old_to_new) 
         & set(test for test in main_data['MAIN2021']) 
         & set(test for test in main_data['MAIN2022']))



### Calculate Demographic Percentages

In [12]:
for year, database in zip(years, MAIN_NAMES):
    current_df = None
    current_df = (main_data[database]['Regents Common Core English Language Art'] 
                  if 'Regents Common Core English Language Art' in main_data[database]
                else main_data[database][new_to_old['Regents Common Core English Language Art']])

    # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
    total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == year)][['ENTITY_CD', 'TESTED', 'YEAR']]
    print("checkpoint 1")

    # Merge the total students for 'All Students' back into the original DataFrame
    current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))
    print("checkpoint 2")

    # List of subgroups of interest
    KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

    # List to store DataFrames
    df_list = []
    columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
    columns = columns.split()

    # Get ENTITY_NAME for 'All Students' subgroup
    multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]
    print("checkpoint 3")

    # Loop over each subgroup and calculate percentage
    for subgroup in KEPT_SUBGROUPS:
        temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == year)].copy()
        subgroup = subgroup.upper()
        temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
        temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
        df_list.append(temp_df)
    print("checkpoint 4")

    # Merge all DataFrames on ENTITY_CD
    result_df = multiple_df
    for temp_df in df_list:
        result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    print("checkpoint 5")


    # Drop observations where TESTED is less than 4
    result_df = result_df[result_df['TESTED_total'] >= 2]
    print("checkpoint 6")

    # Fill NaN values with 0
    result_df = result_df.fillna(0)
    print("checkpoint 7")

    cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
    for col in cols:
        result_df[col] = result_df[col].replace('s', 0)
    print("checkpoint 7")
    if 'Regents Common Core English Language Art' in main_data[database]:
        main_data[database]['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data
    else:
        main_data[database][new_to_old['Regents Common Core English Language Art']] = result_df

checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7


In [13]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2021']:
#     current_df = main_data['MAIN2021']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2021'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2021)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2021)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2021']['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data

In [14]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2022']:
#     current_df = main_data['MAIN2022']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2022'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2022)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2022)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2022']['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data

In [15]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2019']:
#     current_df = main_data['MAIN2019']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2019)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2019)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']] = result_df  # store results in new dictionary instead of main_data

### Concatenate All Dataframes

In [16]:
final_data = pd.concat([
    main_data['MAIN2017'][new_to_old['Regents Common Core English Language Art']],
    main_data['MAIN2018'][new_to_old['Regents Common Core English Language Art']],
    main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']], 
    main_data['MAIN2021']['Regents Common Core English Language Art'], 
    main_data['MAIN2022']['Regents Common Core English Language Art']])
                        

In [17]:
final_data['SUBJECT'] = 'Regents Common Core English Language Art'



In [18]:
# Convert 'YEAR' in final_data to int
final_data['YEAR'] = final_data['YEAR'].astype(int)

# Select specific columns from dropout_df
dropout_subset = dropout_df[['aggregation_code', 'report_school_year', 'dropout_pct']]

# Merge dropout_df with final_data
final_data = pd.merge(final_data, dropout_subset, left_on=['ENTITY_CD', 'YEAR'], right_on=['aggregation_code', 'report_school_year'], how='left')

# Replace '-' with np.nan
final_data['dropout_pct'] = final_data['dropout_pct'].replace('-', np.nan)

# Remove '%' from 'dropout_pct' and convert to float
final_data['dropout_pct'] = final_data['dropout_pct'].str.rstrip('%').astype('float')

final_data['dropout_pct'] = final_data['dropout_pct'].fillna(0)

In [19]:
final_data

,ENTITY_CD,ENTITY_NAME,YEAR,SUBJECT,TESTED,TESTED_total,NUM_LEVEL1,PER_LEVEL1,NUM_LEVEL2,PER_LEVEL2,NUM_LEVEL3,PER_LEVEL3,NUM_LEVEL4,PER_LEVEL4,NUM_LEVEL5,PER_LEVEL5,NUM_PROF,PER_PROF,MALE_PCT,FEMALE_PCT,WHITE_PCT,HISPANIC OR LATINO_PCT,BLACK OR AFRICAN AMERICAN_PCT,ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT,ECONOMICALLY DISADVANTAGED_PCT,aggregation_code,report_school_year,dropout_pct
0,522001040003,STILLWATER MIDDLE SCHOOL HIGH SCHOOL,2017,Regents Common Core English Language Art,103,103.0,11,11,8,08,18,17,18,17,48,47,84.0,81.0,44.660194,55.339806,95.145631,0.970874,1.941748,0.000000,28.155340,NaN,NaN,0.0
1,141901060008,LANCASTER HIGH SCHOOL,2017,Regents Common Core English Language Art,525,525.0,10,02,11,02,52,10,60,11,392,75,504.0,96.0,49.333333,50.666667,93.142857,3.428571,0.761905,0.952381,19.238095,NaN,NaN,0.0
2,580224030008,PATCHOGUE-MEDFORD HIGH SCHOOL,2017,Regents Common Core English Language Art,637,637.0,33,05,33,05,105,16,117,18,349,55,571.0,89.0,51.020408,48.979592,56.671900,33.594976,5.494505,3.610675,48.665620,NaN,NaN,0.0
3,10100010034,ALBANY HIGH SCHOOL,2017,Regents Common Core English Language Art,586,586.0,73,12,50,09,146,25,109,19,208,35,463.0,79.0,50.000000,50.000000,21.160410,17.747440,48.805461,9.726962,51.194539,NaN,NaN,0.0
4,10100860907,GREEN TECH HIGH CHARTER SCHOOL,2017,Regents Common Core English Language Art,91,91.0,10,11,7,08,41,45,18,20,15,16,74.0,81.0,100.000000,0.000000,2.197802,14.285714,78.021978,1.098901,59.340659,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,321000011439,BRONX HIGH SCHOOL-LAW & COMM SVC,2022,Regents Common Core English Language Art,20,20.0,1,05,2,10,7,35,8,40,2,10,17,85,35.000000,65.000000,10.000000,45.000000,40.000000,0.000000,80.000000,3.210000e+11,2022.0,1.0
5770,331300011527,URBAN ASSEMBLY INST OF MATH AND SCIE,2022,Regents Common Core English Language Art,116,116.0,29,25,24,21,38,33,15,13,10,09,63,54,0.000000,100.000000,15.517241,19.827586,47.413793,13.793103,86.206897,3.313000e+11,2022.0,2.0
5771,342500011252,QUEENS SCHOOL OF INQUIRY,2022,Regents Common Core English Language Art,110,110.0,14,13,6,05,24,22,14,13,52,47,90,82,55.454545,44.545455,16.363636,25.454545,11.818182,44.545455,68.181818,3.425000e+11,2022.0,3.0
5772,343000860822,RENAISSANCE CHARTER SCHOOL,2022,Regents Common Core English Language Art,53,53.0,2,04,4,08,12,23,10,19,25,47,47,89,52.830189,47.169811,9.433962,73.584906,3.773585,13.207547,81.132075,3.430009e+11,2022.0,2.0


# Import Virtual Mode Data

In [20]:
virtual = pd.read_csv("../../data/New_York_Schools_LearningModelData_Final.csv", thousands=',')
virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

In [21]:
virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,EnrollmentTotal,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
12,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/3/21,1/9/21,378,Virtual,NaN,NaN,NaN,Remote Only,NaN,NaN,NaN,79.0,0.0,299.0,81.0,45.0
13,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/10/21,1/16/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,237.0,141.0,81.0,81.0
14,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/17/21,1/23/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,226.0,152.0,81.0,76.0
15,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/24/21,1/30/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,229.0,149.0,81.0,81.0
16,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/31/21,2/6/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,230.0,148.0,81.0,80.0


In [22]:
# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter', 'DistrictName'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter', 'DistrictName'], columns='LearningModel', values='Days', fill_value=0)

# # Group by School, Year, and LearningModel and sum the number of days
# grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel'])['Days'].sum().reset_index()

# # Pivot the data so we have separate columns for each learning model
# pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['Hybrid'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']
pivot['HybridPercent'] = pivot['Hybrid'] / pivot['TotalDays']

# Calculate the score for each year
pivot['Score'] = (pivot['Virtual'] + 0.5 * pivot['Hybrid']) / pivot['TotalDays']

# Reset the column names after pivot
pivot.columns.name = None

In [23]:
pivot = pivot.drop(columns=['InPerson', 'Hybrid', 'Virtual', 'TotalDays'])
pivot

,StateAssignedSchoolID,YEAR,Charter,DistrictName,VirtualPercent,HybridPercent,Score
0,10100010014,2021,0,Albany City School District,0.04,0.88,0.48
1,10100010016,2021,0,Albany City School District,0.04,0.88,0.48
2,10100010018,2021,0,Albany City School District,0.04,0.88,0.48
3,10100010019,2021,0,Albany City School District,0.04,0.88,0.48
4,10100010023,2021,0,Albany City School District,0.04,0.88,0.48
...,...,...,...,...,...,...,...
4398,680601060001,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4399,680601060002,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4400,680601060005,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4401,680801040001,2021,0,Dundee Central School District,0.08,0.00,0.08


In [24]:
# Merge dropout_df with final_data
final_data = pd.merge(final_data, pivot, left_on=['ENTITY_CD', 'YEAR'], right_on=['StateAssignedSchoolID', 'YEAR'], how='left')

In [25]:
final_data = final_data.drop(columns=[col for col in "ENTITY_NAME TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF aggregation_code report_school_year StateAssignedSchoolID MALE_PCT FEMALE_PCT dropout_pct SUBJECT".split()])

In [26]:
# Rename multiple columns
final_data = final_data.rename(columns={'ENTITY_CD': 'schoolcode', 
                        'TESTED': 'totalenroll', 
                        'PER_PROF': 'elapass', 
                        'WHITE_PCT': 'white',
                        'HISPANIC OR LATINO_PCT': 'hispanic',     
                        'BLACK OR AFRICAN AMERICAN_PCT': 'black',         
                        'ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT': 'asian',                     
                        'ECONOMICALLY DISADVANTAGED_PCT': 'lowincome',                                         
                        'VirtualPercent': 'virtualper',    
                        'HybridPercent': 'hybridper',    
                        'Score': 'schoolmode',    
                        'YEAR': 'year',    
                        'Charter': "charter",
                        'DistrictName': "district",
                                       })

In [27]:
# Identify 'schoolcode' values of rows in 2021 where 'schoolmode' is NaN
schoolcodes_to_remove = final_data.loc[(final_data['year'] == 2021) & (final_data['schoolmode'].isna()), 'schoolcode'].unique()

# Remove all rows with those 'schoolcode' values
final_data = final_data.loc[~final_data['schoolcode'].isin(schoolcodes_to_remove)]

In [28]:
final_data = final_data.fillna(0)
final_data['district'] = final_data['schoolcode'].astype(str).str[-6:-4].astype(int)
final_data['county'] = final_data['schoolcode'].astype(str).str[-2:]

final_cols = [
'schoolcode',
'district',
'county',
 'year',
 'charter',
 'elapass',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

In [29]:
final_data = final_data[final_cols]
rounding_cols = [
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

final_data[rounding_cols] = final_data[rounding_cols].round(4)

In [30]:
# # Sort the data by school and year
# final_data.sort_values(['schoolcode', 'year'], inplace=True)

# # Replace 0 with NaN in 'district' column
# final_data['district'].replace(0, pd.NA, inplace=True)

# # Now apply the 'ffill' and 'bfill' methods
# final_data['district'] = final_data.groupby('schoolcode')['district'].apply(lambda group: group.ffill().bfill())

# final_data = final_data.dropna(subset=['district'])

In [31]:
first_export = final_data[final_data["year"] != 2022]
first_export

,schoolcode,district,county,year,charter,elapass,schoolmode,virtualper,hybridper,totalenroll,lowincome,white,black,hispanic,asian
0,522001040003,4,03,2017,0.0,81.0,0.00,0.00,0.00,103,28.1553,95.1456,1.9417,0.9709,0.0000
1,141901060008,6,08,2017,0.0,96.0,0.00,0.00,0.00,525,19.2381,93.1429,0.7619,3.4286,0.9524
2,580224030008,3,08,2017,0.0,89.0,0.00,0.00,0.00,637,48.6656,56.6719,5.4945,33.5950,3.6107
3,10100010034,1,34,2017,0.0,79.0,0.00,0.00,0.00,586,51.1945,21.1604,48.8055,17.7474,9.7270
6,10601060008,6,08,2017,0.0,90.0,0.00,0.00,0.00,415,27.9518,67.9518,8.6747,7.4699,11.0843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4561,670401040001,4,01,2021,0.0,92,0.08,0.08,0.00,50,30.0000,92.0000,4.0000,2.0000,2.0000
4562,671201060002,6,02,2021,0.0,100,0.00,0.00,0.00,5,0.0000,80.0000,0.0000,0.0000,20.0000
4563,671501040002,4,02,2021,0.0,94,0.48,0.00,0.96,70,37.1429,95.7143,1.4286,0.0000,1.4286
4564,680601060001,6,01,2021,0.0,90,0.08,0.08,0.00,31,41.9355,87.0968,0.0000,6.4516,0.0000


# Export NYC Data

In [32]:
first_export = final_data[final_data["year"] != 2022]

first_export.to_csv("../final_data_components/data_component_nyc_ela.csv")

In [33]:
# final_data.to_csv("for_running.csv")